# EBM & XGB (ML) Modeling: Financial Ratio Weights 

For our Machine Learning models (‘Explainable Boosting Machine’ (EBM) and ‘Extreme Gradient Boosting’ (XGB)), we ran a model to predict tomorrow's price change for each quarter from 2012 Q1 to 2022 Q1 that contain all health care companies from our original trading on the market during that specific quarter. We split our data into a training set and test set, assigning the first 9 weeks as the training set and the last 3 weeks as the test set per quarter. When running the two models to predict tomorrow's price change, we extracted the feature importance of the ratios variables and it was converted to a standardized weight to be used in the ranking model. 


## 1. Data Preparation

In [45]:
# importing and installing necessary library packages
import pandas as pd
import numpy as np
import itertools  
import matplotlib.pyplot as plt 
import datetime as dt


In [46]:
# reading in data
data = pd.read_csv("final_removenullvalue_Jun27.csv")

# sorting data
data = data.sort_values(by="date")

# converting date data type to datetime
data['date'] = pd.to_datetime(data['date'])

# setting a colmn as the index
data = data.set_index('date')

data


,Unnamed: 0,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,...,inventory,totalCurrentAssets,totalAssets,totalCurrentLiabilities,totalLiabilities,totalStockholdersEquity,totalEquity,operatingCashFlow,Annual EPS Growth,PEG Ratio
date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,619261,0.636364,0.636364,0.636364,0.636364,0.318023,880,880,0.00000,0.000,...,5.580000e+06,1.965500e+07,2.388800e+07,1.132200e+07,1.322400e+07,1.066400e+07,1.066400e+07,-7.700000e+05,0.000001,1.930111e+07
2012-01-03,55821,18.000000,18.719999,16.920000,17.280001,17.280001,121347,121347,-0.72000,-4.000,...,0.000000e+00,5.019065e+07,5.457492e+07,6.084953e+06,1.657854e+07,3.799638e+07,3.799638e+07,-7.689313e+06,0.000001,-1.925612e+06
2012-01-03,371621,25.841973,26.305873,25.794219,26.128500,26.128500,18100427,18100427,0.28653,1.109,...,6.339000e+09,3.434400e+10,1.055090e+11,1.658200e+10,4.819500e+10,5.486000e+10,5.731400e+10,2.154000e+09,0.000001,1.603309e+07
2012-01-03,105081,34.480000,35.600000,34.230000,34.930000,34.930000,1215200,1215200,0.45000,1.305,...,1.240640e+08,5.133900e+08,1.297085e+09,1.198140e+08,5.251350e+08,7.719500e+08,7.719500e+08,-7.171000e+06,0.000001,-4.110157e+07
2012-01-03,202445,33.700001,33.900002,32.700001,32.799999,32.799999,143280,143280,-0.90000,-2.671,...,0.000000e+00,1.101240e+08,1.206760e+08,2.581700e+07,3.229700e+07,8.837900e+07,8.837900e+07,-8.181000e+06,0.000001,-1.184214e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-31,590178,39.100000,39.790000,38.050000,38.220000,38.220000,472559,472559,-0.88000,-2.251,...,1.438500e+07,1.518800e+08,2.402880e+08,2.173100e+07,6.780300e+07,1.724850e+08,1.724850e+08,3.468000e+06,0.000001,-6.331148e+07
2022-03-31,208812,42.360000,42.360000,40.740000,40.740000,40.740000,252200,252200,-1.62000,-3.824,...,1.345000e+09,3.194000e+09,1.625100e+10,1.307000e+09,8.639000e+09,7.612000e+09,7.612000e+09,-6.200000e+07,0.000001,1.035475e+08
2022-03-31,314577,16.350000,16.840000,16.210000,16.270000,16.270000,771699,771699,-0.08000,-0.489,...,0.000000e+00,4.773530e+08,5.033570e+08,1.510360e+08,8.751180e+08,-3.717610e+08,-3.717610e+08,-2.012400e+07,0.000001,-6.988456e+06


In [47]:
# selecting certain columns
data = data[['adjClose','Quarter','Ticker','priceToBookRatio','priceToSalesRatio','priceEarningsRatio','priceToFreeCashFlowsRatio','priceToOperatingCashFlowsRatio','priceCashFlowRatio','grossProfitMargin','operatingProfitMargin','pretaxProfitMargin','netProfitMargin','returnOnEquity','currentRatio','quickRatio','cashRatio','debtEquityRatio','eps', 'PEG Ratio']]


In [60]:
# creating a column for percent change of daily prices
groups = data.groupby(['Ticker'])
data['pct_chg'] = groups['adjClose'].pct_change()

data


C:\Users\zzxxc\AppData\Local\Temp/ipykernel_12368/1926667512.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['pct_chg'] = groups['adjClose'].pct_change()


,adjClose,Quarter,Ticker,priceToBookRatio,priceToSalesRatio,priceEarningsRatio,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,grossProfitMargin,...,PEG Ratio,pct_chg,date,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear
date,,,,,,,,,,,,,,,,,,,,,
2012-01-04,4.130000,2012Q1,AMN,1.749053,1.086031,14.376202,28.752404,25.739617,25.739617,0.279199,...,1.437620e+07,NaN,2012-01-04,2,1,1,2012,4,4,1
2012-01-04,32.000000,2012Q1,SMTI,-3.880208,135.139178,12.236698,-21.431941,-21.431941,-21.431941,-0.136422,...,1.223670e+07,NaN,2012-01-04,2,1,1,2012,4,4,1
2012-01-04,9.560000,2012Q1,ANIK,1.679837,11.494027,21.581007,-86.252350,-97.690001,-97.690001,0.553399,...,2.158101e+07,NaN,2012-01-04,2,1,1,2012,4,4,1
2012-01-04,14.510000,2012Q1,ANGO,0.795355,6.344747,-46.264081,56.723228,48.192603,48.192603,0.570404,...,-4.626408e+07,NaN,2012-01-04,2,1,1,2012,4,4,1
2012-01-04,30.671413,2012Q1,LLY,3.032207,8.029286,11.121566,62.316514,52.762533,52.762533,0.786166,...,1.112157e+07,NaN,2012-01-04,2,1,1,2012,4,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-31,38.220000,2022Q1,VCEL,10.411147,49.780083,-63.311476,4640.224031,517.810467,517.810467,0.650108,...,-6.331148e+07,-0.025497,2022-03-31,3,1,3,2022,90,31,13
2022-03-31,40.740000,2022Q1,ELAT,2.611813,16.229486,103.547500,-225.921818,-320.663226,-320.663226,0.584490,...,1.035475e+08,-0.040057,2022-03-31,3,1,3,2022,90,31,13
2022-03-31,16.270000,2022Q1,ICPT,-1.299636,5.454313,-6.988456,-24.000493,-24.008841,-24.008841,0.991443,...,-6.988456e+06,0.001847,2022-03-31,3,1,3,2022,90,31,13


In [35]:
# renaming a column
data = data.rename(columns = {'adjClose':'Actual_Close'})


In [36]:
# selecting certain columns
data = data[['pct_chg', 'Actual_Close', 'Ticker','Quarter']]
data


In [37]:
# shfting data by a period 
# (eg. skip the first period in the data '2012-01-03' and rearrange data starting from '2012-01-04')  
# (the first period in the dataset will display 'NaN' as a result) 
data3 = data2.groupby(by=['Ticker']).shift(1)
data3


In [51]:
# renaming a column
data2 = data2.rename(columns = {'Ticker2':'Ticker'})
data2


,adjClose,Quarter,Ticker,priceToBookRatio,priceToSalesRatio,priceEarningsRatio,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,grossProfitMargin,...,pretaxProfitMargin,netProfitMargin,returnOnEquity,currentRatio,quickRatio,cashRatio,debtEquityRatio,eps,PEG Ratio,Ticker
date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,0.318023,2012Q1,ZYXI,2.316713,2.762234,19.301113,-25.601477,-32.084968,-32.084968,0.797294,...,0.043940,0.035778,0.030008,1.736001,1.093800,0.065536,1.240060,0.009091,1.930111e+07,ZYXI
2012-01-03,17.280001,2012Q1,ANIP,2.080767,693.522909,-1.925612,-10.210000,-10.282012,-10.282012,1.000000,...,-90.039272,-90.039272,-0.270144,8.248322,8.130504,8.130504,0.436319,-3.239987,-1.925612e+06,ANIP
2012-01-03,26.128500,2012Q1,MRK,2.031754,9.501493,16.033086,61.142082,51.746526,51.746526,0.655869,...,0.213707,0.148154,0.031681,2.071162,1.464962,0.883850,0.878509,0.571338,1.603309e+07,MRK
2012-01-03,34.930000,2012Q1,BMRN,5.105444,33.786381,-41.101572,-295.217041,-549.595245,-549.595245,0.853364,...,-0.205797,-0.205505,-0.031054,4.284892,2.827775,0.689285,0.680271,-0.210000,-4.110157e+07,BMRN
2012-01-03,32.799999,2012Q1,DVAX,8.846196,332.688474,-11.842137,-85.791497,-95.565079,-95.565079,1.000000,...,0.000000,-7.023404,-0.186753,4.265561,4.193206,0.765116,0.365437,-1.100000,-1.184214e+07,DVAX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-31,38.220000,2022Q1,VCEL,10.411147,49.780083,-63.311476,4640.224031,517.810467,517.810467,0.650108,...,-0.196568,-0.196568,-0.041111,6.989094,6.092771,2.561272,0.393095,-0.150000,-6.331148e+07,VCEL
2022-03-31,40.740000,2022Q1,ELAT,2.611813,16.229486,103.547500,-225.921818,-320.663226,-320.663226,0.584490,...,0.057959,0.039184,0.006306,2.443764,1.173680,0.261668,1.134919,0.098000,1.035475e+08,ELAT
2022-03-31,16.270000,2022Q1,ICPT,-1.299636,5.454313,-6.988456,-24.000493,-24.008841,-24.008841,0.991443,...,-0.195119,-0.195119,0.046492,3.160525,2.925435,0.319924,-2.353980,-0.580000,-6.988456e+06,ICPT


In [44]:
data2[['pct_chg', 'Actual_Close', 'Ticker', 'Quarter']] = data[['pct_chg', 'Actual_Close', 'Ticker', 'Quarter']]


In [15]:
data = data2


In [52]:
# dropping null values
data = data.dropna()
data


,adjClose,Quarter,Ticker,priceToBookRatio,priceToSalesRatio,priceEarningsRatio,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,grossProfitMargin,...,pretaxProfitMargin,netProfitMargin,returnOnEquity,currentRatio,quickRatio,cashRatio,debtEquityRatio,eps,PEG Ratio,pct_chg
date,,,,,,,,,,,,,,,,,,,,,
2012-01-04,4.130000,2012Q1,AMN,1.749053,1.086031,14.376202,28.752404,25.739617,25.739617,0.279199,...,0.030078,0.018886,0.030416,1.912681,1.643675,0.046440,2.727019,0.090000,1.437620e+07,-0.019002
2012-01-04,32.000000,2012Q1,SMTI,-3.880208,135.139178,12.236698,-21.431941,-21.431941,-21.431941,-0.136422,...,2.760940,2.760940,-0.079274,0.176510,0.103075,0.000227,-1.901425,0.490328,1.223670e+07,-0.179487
2012-01-04,9.560000,2012Q1,ANIK,1.679837,11.494027,21.581007,-86.252350,-97.690001,-97.690001,0.553399,...,0.209730,0.133150,0.019460,4.965181,3.960024,2.639953,0.355869,0.150000,2.158101e+07,0.026853
2012-01-04,14.510000,2012Q1,ANGO,0.795355,6.344747,-46.264081,56.723228,48.192603,48.192603,0.570404,...,-0.055850,-0.034285,-0.004298,7.229191,5.814551,1.166233,0.085676,-0.070000,-4.626408e+07,-0.020918
2012-01-04,30.671413,2012Q1,LLY,3.032207,8.029286,11.121566,62.316514,52.762533,52.762533,0.786166,...,0.238361,0.180489,0.068161,1.922975,1.351282,0.628988,1.174025,0.910000,1.112157e+07,-0.026077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-31,38.220000,2022Q1,VCEL,10.411147,49.780083,-63.311476,4640.224031,517.810467,517.810467,0.650108,...,-0.196568,-0.196568,-0.041111,6.989094,6.092771,2.561272,0.393095,-0.150000,-6.331148e+07,-0.025497
2022-03-31,40.740000,2022Q1,ELAT,2.611813,16.229486,103.547500,-225.921818,-320.663226,-320.663226,0.584490,...,0.057959,0.039184,0.006306,2.443764,1.173680,0.261668,1.134919,0.098000,1.035475e+08,-0.040057
2022-03-31,16.270000,2022Q1,ICPT,-1.299636,5.454313,-6.988456,-24.000493,-24.008841,-24.008841,0.991443,...,-0.195119,-0.195119,0.046492,3.160525,2.925435,0.319924,-2.353980,-0.580000,-6.988456e+06,0.001847


In [53]:
# creating columns
data['date'] = data.index
data['dayofweek'] = data['date'].dt.dayofweek
data['quarter'] = data['date'].dt.quarter
data['month'] = data['date'].dt.month
data['year'] = data['date'].dt.year
data['dayofyear'] = data['date'].dt.dayofyear
data['dayofmonth'] = data['date'].dt.day
data['weekofyear'] = data['date'].dt.weekofyear

data


C:\Users\zzxxc\AppData\Local\Temp/ipykernel_12368/3017859989.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['date'] = data.index
C:\Users\zzxxc\AppData\Local\Temp/ipykernel_12368/3017859989.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['dayofweek'] = data['date'].dt.dayofweek
C:\Users\zzxxc\AppData\Local\Temp/ipykernel_12368/3017859989.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

,adjClose,Quarter,Ticker,priceToBookRatio,priceToSalesRatio,priceEarningsRatio,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,grossProfitMargin,...,PEG Ratio,pct_chg,date,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear
date,,,,,,,,,,,,,,,,,,,,,
2012-01-04,4.130000,2012Q1,AMN,1.749053,1.086031,14.376202,28.752404,25.739617,25.739617,0.279199,...,1.437620e+07,-0.019002,2012-01-04,2,1,1,2012,4,4,1
2012-01-04,32.000000,2012Q1,SMTI,-3.880208,135.139178,12.236698,-21.431941,-21.431941,-21.431941,-0.136422,...,1.223670e+07,-0.179487,2012-01-04,2,1,1,2012,4,4,1
2012-01-04,9.560000,2012Q1,ANIK,1.679837,11.494027,21.581007,-86.252350,-97.690001,-97.690001,0.553399,...,2.158101e+07,0.026853,2012-01-04,2,1,1,2012,4,4,1
2012-01-04,14.510000,2012Q1,ANGO,0.795355,6.344747,-46.264081,56.723228,48.192603,48.192603,0.570404,...,-4.626408e+07,-0.020918,2012-01-04,2,1,1,2012,4,4,1
2012-01-04,30.671413,2012Q1,LLY,3.032207,8.029286,11.121566,62.316514,52.762533,52.762533,0.786166,...,1.112157e+07,-0.026077,2012-01-04,2,1,1,2012,4,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-31,38.220000,2022Q1,VCEL,10.411147,49.780083,-63.311476,4640.224031,517.810467,517.810467,0.650108,...,-6.331148e+07,-0.025497,2022-03-31,3,1,3,2022,90,31,13
2022-03-31,40.740000,2022Q1,ELAT,2.611813,16.229486,103.547500,-225.921818,-320.663226,-320.663226,0.584490,...,1.035475e+08,-0.040057,2022-03-31,3,1,3,2022,90,31,13
2022-03-31,16.270000,2022Q1,ICPT,-1.299636,5.454313,-6.988456,-24.000493,-24.008841,-24.008841,0.991443,...,-6.988456e+06,0.001847,2022-03-31,3,1,3,2022,90,31,13


## 2. Modeling
##### 2-1. EBM 

In [54]:
from interpret.glassbox import ExplainableBoostingRegressor  # interpret ebm class
from interpret.perf import RegressionPerf

def ebm_grid(train, valid, x_names, y_name, gs_params=None, n_models=None, early_stopping_rounds=None, seed=None):
    
    """ the self defined fucntion above performs a random grid search over 'n_models' and 'gs_params.'
    
    :parameter 'train': Training data as a Pandas DataFrame.
    :parameter 'valid': Validation data as a Pandas DataFrame.
    :parameter 'x_names': Names of the input (independent or predictor) variables.
    :parameter 'y_name': Name of the target variable.
    :parameter 'gs_params': Dictionary of the lists of potential EBM parameters over which to search.   
    :parameter 'n_models': Number of random models to evaluate.
    :parameter 'early_stopping_rounds': EBM early stopping rounds.
    :parameter 'seed': Random seed for better interpretability.
    :return: Best candidate model from random grid search.

    """

    # cartesian product of gs_params
    keys, values = zip(*gs_params.items())
    experiments = [dict(zip(keys, v)) for v in itertools.product(*values)]
    

    # preserving exact reproducibility for this function
    np.random.seed(SEED) 
    
    # selecting randomly from cartesian product space
    selected_experiments = np.random.choice(len(experiments), n_models)

    # setting global parameters for seed, etc.
    params = {'n_jobs': NTHREAD,
              'early_stopping_rounds': early_stopping_rounds, 
              'random_state': SEED}

    # initial grid search loop
    best_candidate = None
    best_score = -1

    # grid search loop
    for i, exp in enumerate(selected_experiments):

        params.update(experiments[exp])  # override global params with current grid run params

        print('Grid search run %d/%d:' % (int(i + 1), int(n_models)))
        print('Training with parameters:', params)
        
        # training
        ebm = ExplainableBoostingRegressor(**params)
        candidate = ebm.fit(train[x_names], train[y_name]) 
        
        # calculating R^2
        ebm_perf = RegressionPerf(ebm.predict).explain_perf(valid[x_names], valid[y_name], name='EBM')       
        candidate_best_score = ebm_perf._internal_obj['overall']['r^2']
        

        # determining if current model is better than the best previous one
        if candidate_best_score > best_score:
            best_candidate = candidate
            best_score = candidate_best_score
            print('Grid search new best score discovered at iteration %d/%d: %.4f.' %
                             (int(i + 1), int(n_models), candidate_best_score))

        print('---------- ----------')
        
        del ebm
            
    return best_candidate

# P.S. It takes a while to run this code 👇 - 25 mins

In [55]:
SEED = 12345
NTHREAD = 4
np.random.seed(SEED) # preserving exact reproducibility for this cell

weights_ebm = pd.DataFrame()
all_ebm_phat = pd.DataFrame()
quarters = data['Quarter'].unique()

for j in quarters: 
     
    data_q =  data[data['Quarter'] == j]

    # getting the first and the last datetime for the final week of data
    range_max = data_q['date'].max()
    range_min = range_max - dt.timedelta(days=21)

    # taking slice with the final week of data
    train = data_q[data_q['date'] < range_min]
    valid = data_q[(data_q['date'] >= range_min) & 
                   (data_q['date'] <= range_max)]

    x_names = ['adjClose', 'dayofweek','quarter','month','year','dayofyear','dayofmonth','weekofyear', 'Ticker','priceToBookRatio','priceToSalesRatio','priceEarningsRatio','priceToFreeCashFlowsRatio','priceToOperatingCashFlowsRatio','priceCashFlowRatio','grossProfitMargin','operatingProfitMargin','pretaxProfitMargin','netProfitMargin','returnOnEquity','currentRatio','quickRatio','cashRatio','debtEquityRatio', 'eps','PEG Ratio']
    ratios = ['priceToBookRatio','priceToSalesRatio','priceEarningsRatio','priceToFreeCashFlowsRatio','priceToOperatingCashFlowsRatio','priceCashFlowRatio','grossProfitMargin','operatingProfitMargin','pretaxProfitMargin','netProfitMargin','returnOnEquity','currentRatio','quickRatio','cashRatio','debtEquityRatio', 'eps','PEG Ratio']
    y_name = 'pct_chg'


    # dictionary of hyperparameter values in the lists for grid search
    gs_params = {'max_bins': [128, 256, 512],
             'max_interaction_bins': [16, 32, 64],
             'interactions': [5, 10, 15],
             'outer_bags': [4, 8, 12], 
             'inner_bags': [0, 4],
             'learning_rate': [0.001, 0.01, 0.05],
             'validation_size': [0.1, 0.25, 0.5],
             'min_samples_leaf': [1, 2, 5, 10],
             'max_leaves': [1, 3, 5]}

    # starting local timer
    #ebm_tic = time.time()

    # EBM grid search
    best_ebm = ebm_grid(train, valid, x_names, y_name, gs_params=gs_params, n_models=10, 
                    early_stopping_rounds=100, seed=SEED)


    best_ebm_perf = RegressionPerf(best_ebm.predict).explain_perf(valid[x_names], valid[y_name], name='EBM')

    r2=(best_ebm_perf._internal_obj['overall']['r^2'])

    # calculating predictions
    ebm_phat = pd.DataFrame(best_ebm.predict(valid[x_names]), columns=['phat'])
    ebm_phat = pd.concat([data.reset_index(drop=True), ebm_phat], axis=1)
    all_ebm_phat =  all_ebm_phat.append(ebm_phat)

    best_ebm_phat = pd.DataFrame(best_ebm.predict(valid[x_names]), columns=['phat']) 
    best_ebm_phat = pd.concat([valid.reset_index(drop=True), best_ebm_phat], axis=1)
    best_ebm_contribs = best_ebm.explain_local(valid[x_names])
    names = best_ebm_contribs._internal_obj['specific'][0]['names']

    best_ebm_contribs_df = pd.DataFrame(columns=names)

    # extract scores from insane _internal_obj
    # new versions just have predict_and_explain() function
    # use that if you can
    for l in range(0, valid.shape[0]):
        row_df = pd.DataFrame(np.array(best_ebm_contribs._internal_obj['specific'][l]['scores']).reshape(1, len(names)), columns=names)
        best_ebm_contribs_df = best_ebm_contribs_df.append(row_df, ignore_index=True)
        del row_df

    global_fi = pd.DataFrame()
    global_fi['EBM Importance'] = np.abs(best_ebm_contribs_df).mean()
    global_fi['ratio'] = global_fi.index
    im= global_fi[global_fi['ratio'].isin(ratios)].sort_values(by='EBM Importance', ascending=False)
    im['weights'] = im['EBM Importance'] / (im['EBM Importance'].sum())
    im['Quarter'] = j
    weights_ebm = weights_ebm.append(im)


Grid search run 1/10:
Training with parameters: {'n_jobs': 4, 'early_stopping_rounds': 100, 'random_state': 12345, 'max_bins': 512, 'max_interaction_bins': 16, 'interactions': 5, 'outer_bags': 4, 'inner_bags': 0, 'learning_rate': 0.05, 'validation_size': 0.25, 'min_samples_leaf': 1, 'max_leaves': 3}
Grid search new best score discovered at iteration 1/10: 0.0164.
---------- ----------
Grid search run 2/10:
Training with parameters: {'n_jobs': 4, 'early_stopping_rounds': 100, 'random_state': 12345, 'max_bins': 128, 'max_interaction_bins': 32, 'interactions': 5, 'outer_bags': 8, 'inner_bags': 0, 'learning_rate': 0.001, 'validation_size': 0.25, 'min_samples_leaf': 2, 'max_leaves': 5}
---------- ----------
Grid search run 3/10:
Training with parameters: {'n_jobs': 4, 'early_stopping_rounds': 100, 'random_state': 12345, 'max_bins': 512, 'max_interaction_bins': 16, 'interactions': 5, 'outer_bags': 4, 'inner_bags': 0, 'learning_rate': 0.001, 'validation_size': 0.5, 'min_samples_leaf': 1, 'max

---------- ----------
Grid search run 5/10:
Training with parameters: {'n_jobs': 4, 'early_stopping_rounds': 100, 'random_state': 12345, 'max_bins': 512, 'max_interaction_bins': 64, 'interactions': 15, 'outer_bags': 4, 'inner_bags': 0, 'learning_rate': 0.05, 'validation_size': 0.1, 'min_samples_leaf': 10, 'max_leaves': 3}
---------- ----------
Grid search run 6/10:
Training with parameters: {'n_jobs': 4, 'early_stopping_rounds': 100, 'random_state': 12345, 'max_bins': 256, 'max_interaction_bins': 16, 'interactions': 15, 'outer_bags': 12, 'inner_bags': 4, 'learning_rate': 0.01, 'validation_size': 0.1, 'min_samples_leaf': 2, 'max_leaves': 5}
---------- ----------
Grid search run 7/10:
Training with parameters: {'n_jobs': 4, 'early_stopping_rounds': 100, 'random_state': 12345, 'max_bins': 512, 'max_interaction_bins': 32, 'interactions': 15, 'outer_bags': 4, 'inner_bags': 4, 'learning_rate': 0.05, 'validation_size': 0.25, 'min_samples_leaf': 10, 'max_leaves': 1}
Grid search new best score 

---------- ----------
Grid search run 10/10:
Training with parameters: {'n_jobs': 4, 'early_stopping_rounds': 100, 'random_state': 12345, 'max_bins': 512, 'max_interaction_bins': 64, 'interactions': 5, 'outer_bags': 4, 'inner_bags': 0, 'learning_rate': 0.001, 'validation_size': 0.25, 'min_samples_leaf': 2, 'max_leaves': 3}
---------- ----------
Grid search run 1/10:
Training with parameters: {'n_jobs': 4, 'early_stopping_rounds': 100, 'random_state': 12345, 'max_bins': 512, 'max_interaction_bins': 16, 'interactions': 5, 'outer_bags': 4, 'inner_bags': 0, 'learning_rate': 0.05, 'validation_size': 0.25, 'min_samples_leaf': 1, 'max_leaves': 3}
Grid search new best score discovered at iteration 1/10: -0.0539.
---------- ----------
Grid search run 2/10:
Training with parameters: {'n_jobs': 4, 'early_stopping_rounds': 100, 'random_state': 12345, 'max_bins': 128, 'max_interaction_bins': 32, 'interactions': 5, 'outer_bags': 8, 'inner_bags': 0, 'learning_rate': 0.001, 'validation_size': 0.25, 'm

Grid search new best score discovered at iteration 3/10: -0.0084.
---------- ----------
Grid search run 4/10:
Training with parameters: {'n_jobs': 4, 'early_stopping_rounds': 100, 'random_state': 12345, 'max_bins': 128, 'max_interaction_bins': 64, 'interactions': 5, 'outer_bags': 4, 'inner_bags': 4, 'learning_rate': 0.05, 'validation_size': 0.5, 'min_samples_leaf': 1, 'max_leaves': 5}
---------- ----------
Grid search run 5/10:
Training with parameters: {'n_jobs': 4, 'early_stopping_rounds': 100, 'random_state': 12345, 'max_bins': 512, 'max_interaction_bins': 64, 'interactions': 15, 'outer_bags': 4, 'inner_bags': 0, 'learning_rate': 0.05, 'validation_size': 0.1, 'min_samples_leaf': 10, 'max_leaves': 3}
---------- ----------
Grid search run 6/10:
Training with parameters: {'n_jobs': 4, 'early_stopping_rounds': 100, 'random_state': 12345, 'max_bins': 256, 'max_interaction_bins': 16, 'interactions': 15, 'outer_bags': 12, 'inner_bags': 4, 'learning_rate': 0.01, 'validation_size': 0.1, 'min

---------- ----------
Grid search run 8/10:
Training with parameters: {'n_jobs': 4, 'early_stopping_rounds': 100, 'random_state': 12345, 'max_bins': 256, 'max_interaction_bins': 16, 'interactions': 15, 'outer_bags': 8, 'inner_bags': 4, 'learning_rate': 0.001, 'validation_size': 0.5, 'min_samples_leaf': 5, 'max_leaves': 3}
Grid search new best score discovered at iteration 8/10: -0.0004.
---------- ----------
Grid search run 9/10:
Training with parameters: {'n_jobs': 4, 'early_stopping_rounds': 100, 'random_state': 12345, 'max_bins': 256, 'max_interaction_bins': 16, 'interactions': 10, 'outer_bags': 8, 'inner_bags': 0, 'learning_rate': 0.05, 'validation_size': 0.5, 'min_samples_leaf': 5, 'max_leaves': 1}
---------- ----------
Grid search run 10/10:
Training with parameters: {'n_jobs': 4, 'early_stopping_rounds': 100, 'random_state': 12345, 'max_bins': 512, 'max_interaction_bins': 64, 'interactions': 5, 'outer_bags': 4, 'inner_bags': 0, 'learning_rate': 0.001, 'validation_size': 0.25, 'm

...

---------- ----------
Grid search run 5/10:
Training with parameters: {'n_jobs': 4, 'early_stopping_rounds': 100, 'random_state': 12345, 'max_bins': 512, 'max_interaction_bins': 64, 'interactions': 15, 'outer_bags': 4, 'inner_bags': 0, 'learning_rate': 0.05, 'validation_size': 0.1, 'min_samples_leaf': 10, 'max_leaves': 3}
---------- ----------
Grid search run 6/10:
Training with parameters: {'n_jobs': 4, 'early_stopping_rounds': 100, 'random_state': 12345, 'max_bins': 256, 'max_interaction_bins': 16, 'interactions': 15, 'outer_bags': 12, 'inner_bags': 4, 'learning_rate': 0.01, 'validation_size': 0.1, 'min_samples_leaf': 2, 'max_leaves': 5}
---------- ----------
Grid search run 7/10:
Training with parameters: {'n_jobs': 4, 'early_stopping_rounds': 100, 'random_state': 12345, 'max_bins': 512, 'max_interaction_bins': 32, 'interactions': 15, 'outer_bags': 4, 'inner_bags': 4, 'learning_rate': 0.05, 'validation_size': 0.25, 'min_samples_leaf': 10, 'max_leaves': 1}
---------- ----------
Grid 

Grid search new best score discovered at iteration 8/10: -0.0151.
---------- ----------
Grid search run 9/10:
Training with parameters: {'n_jobs': 4, 'early_stopping_rounds': 100, 'random_state': 12345, 'max_bins': 256, 'max_interaction_bins': 16, 'interactions': 10, 'outer_bags': 8, 'inner_bags': 0, 'learning_rate': 0.05, 'validation_size': 0.5, 'min_samples_leaf': 5, 'max_leaves': 1}
---------- ----------
Grid search run 10/10:
Training with parameters: {'n_jobs': 4, 'early_stopping_rounds': 100, 'random_state': 12345, 'max_bins': 512, 'max_interaction_bins': 64, 'interactions': 5, 'outer_bags': 4, 'inner_bags': 0, 'learning_rate': 0.001, 'validation_size': 0.25, 'min_samples_leaf': 2, 'max_leaves': 3}
---------- ----------
Grid search run 1/10:
Training with parameters: {'n_jobs': 4, 'early_stopping_rounds': 100, 'random_state': 12345, 'max_bins': 512, 'max_interaction_bins': 16, 'interactions': 5, 'outer_bags': 4, 'inner_bags': 0, 'learning_rate': 0.05, 'validation_size': 0.25, 'mi

Grid search new best score discovered at iteration 2/10: -0.0263.
---------- ----------
Grid search run 3/10:
Training with parameters: {'n_jobs': 4, 'early_stopping_rounds': 100, 'random_state': 12345, 'max_bins': 512, 'max_interaction_bins': 16, 'interactions': 5, 'outer_bags': 4, 'inner_bags': 0, 'learning_rate': 0.001, 'validation_size': 0.5, 'min_samples_leaf': 1, 'max_leaves': 3}
---------- ----------
Grid search run 4/10:
Training with parameters: {'n_jobs': 4, 'early_stopping_rounds': 100, 'random_state': 12345, 'max_bins': 128, 'max_interaction_bins': 64, 'interactions': 5, 'outer_bags': 4, 'inner_bags': 4, 'learning_rate': 0.05, 'validation_size': 0.5, 'min_samples_leaf': 1, 'max_leaves': 5}
---------- ----------
Grid search run 5/10:
Training with parameters: {'n_jobs': 4, 'early_stopping_rounds': 100, 'random_state': 12345, 'max_bins': 512, 'max_interaction_bins': 64, 'interactions': 15, 'outer_bags': 4, 'inner_bags': 0, 'learning_rate': 0.05, 'validation_size': 0.1, 'min_s

---------- ----------
Grid search run 7/10:
Training with parameters: {'n_jobs': 4, 'early_stopping_rounds': 100, 'random_state': 12345, 'max_bins': 512, 'max_interaction_bins': 32, 'interactions': 15, 'outer_bags': 4, 'inner_bags': 4, 'learning_rate': 0.05, 'validation_size': 0.25, 'min_samples_leaf': 10, 'max_leaves': 1}
Grid search new best score discovered at iteration 7/10: -0.0087.
---------- ----------
Grid search run 8/10:
Training with parameters: {'n_jobs': 4, 'early_stopping_rounds': 100, 'random_state': 12345, 'max_bins': 256, 'max_interaction_bins': 16, 'interactions': 15, 'outer_bags': 8, 'inner_bags': 4, 'learning_rate': 0.001, 'validation_size': 0.5, 'min_samples_leaf': 5, 'max_leaves': 3}
---------- ----------
Grid search run 9/10:
Training with parameters: {'n_jobs': 4, 'early_stopping_rounds': 100, 'random_state': 12345, 'max_bins': 256, 'max_interaction_bins': 16, 'interactions': 10, 'outer_bags': 8, 'inner_bags': 0, 'learning_rate': 0.05, 'validation_size': 0.5, 'm

Grid search new best score discovered at iteration 10/10: -0.0075.
---------- ----------


In [56]:
# viewing 'weights_ebm'
weights_ebm


,EBM Importance,ratio,weights,Quarter
returnOnEquity,0.000491,returnOnEquity,0.093452,2012Q1
grossProfitMargin,0.000458,grossProfitMargin,0.087204,2012Q1
operatingProfitMargin,0.000355,operatingProfitMargin,0.067562,2012Q1
cashRatio,0.000338,cashRatio,0.064264,2012Q1
eps,0.000336,eps,0.064068,2012Q1
...,...,...,...,...
operatingProfitMargin,0.000047,operatingProfitMargin,0.046030,2022Q1
priceToSalesRatio,0.000047,priceToSalesRatio,0.045976,2022Q1
debtEquityRatio,0.000040,debtEquityRatio,0.038877,2022Q1
netProfitMargin,0.000039,netProfitMargin,0.038130,2022Q1


In [15]:
# saving the data in the csv format
weights_ebm.to_csv("weights_ebm_j25.csv")


##### 2-2. xgb

In [57]:
import xgboost as xgb
def xgb_grid(dtrain, dvalid, gs_params=None, n_models=None,
             ntree=None, early_stopping_rounds=None, verbose=False, seed=None):
    
    """ the self defined fucntion above performs performs a random grid search over 'n_models' and 'gs_params.'

    :parameter 'dtrain': Training data in LightSVM format.
    :parameter 'dvalid': Validation data in LightSVM format.
    :parameter 'mono_constraints': User-supplied monotonicity constraints.
    :parameter 'gs_params': Dictionary of lists of potential XGBoost parameters over which to search.
    :parameter 'n_models': Number of random models to evaluate.
    :parameter 'ntree': Number of trees in XGBoost model.
    :parameter 'early_stopping_rounds': XGBoost early stopping rounds.
    :parameter 'verbose': Whether to display training iterations, default False.
    :parameter 'seed': Random seed for better interpretability.
    :return: Best candidate model from random grid search.

    """

    # cartesian product of gs_params
    keys, values = zip(*gs_params.items())
    experiments = [dict(zip(keys, v)) for v in itertools.product(*values)]

    # preserving exact reproducibility for this function
    np.random.seed(SEED) 
    
    # selecting randomly from cartesian product space
    selected_experiments = np.random.choice(len(experiments), n_models)

    # setting global params for objective,  etc.
    params = {'booster': 'gbtree',
              'eval_metric': 'rmse',
              'nthread': NTHREAD,
              'objective': 'reg:squarederror',
              'seed': SEED}

    # initial grid search loop
    best_candidate = None
    best_score = 0

    # grid search loop
    for i, exp in enumerate(selected_experiments):

        params.update(experiments[exp])  # override global params with current grid run params

        print('Grid search run %d/%d:' % (int(i + 1), int(n_models)))
        print('Training with parameters:', params)

        # train on current params
        watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
        
        candidate = xgb.train(params,
                              dtrain,
                              ntree,
                              early_stopping_rounds=early_stopping_rounds,
                              evals=watchlist,
                              verbose_eval=verbose)    

        # determining if current model is better than the previous best
        if candidate.best_score > best_score:
            best_candidate = candidate
            best_score = candidate.best_score
            print('Grid search new best score discovered at iteration %d/%d: %.4f.' %
                             (int(i + 1), int(n_models), candidate.best_score))

        print('---------- ----------')
            
    return best_candidate

# P.S. It takes a while to run this code 👇

In [58]:
all_xgb_phat = pd.DataFrame()
weights_xgb = pd.DataFrame()
SEED = 12345
NTHREAD = 4
np.random.seed(SEED) # preserving exact reproducibility for this cell

quarters = data['Quarter'].unique()


for j in quarters:

    data_q =  data[data['Quarter'] == j]
    from sklearn import preprocessing
    lbl = preprocessing.LabelEncoder()
    data_q['Ticker'] = lbl.fit_transform(data_q['Ticker'].astype(str))
    
    # executing split
    # getting the first and the last datetime for the final week of data
    range_max = data_q['date'].max()
    range_min = range_max - dt.timedelta(days=21)

    # takeing slice with the final week of data
    train = data_q[data_q['date'] < range_min]
    valid = data_q[(data_q['date'] >= range_min) & 
                   (data_q['date'] <= range_max)]


    x_names = ['adjClose', 'dayofweek','quarter','month','year','dayofyear','dayofmonth','weekofyear', 'Ticker', 'priceToBookRatio','priceToSalesRatio','priceEarningsRatio','priceToFreeCashFlowsRatio','priceToOperatingCashFlowsRatio','priceCashFlowRatio','grossProfitMargin','operatingProfitMargin','pretaxProfitMargin','netProfitMargin','returnOnEquity','currentRatio','quickRatio','cashRatio','debtEquityRatio', 'eps','PEG Ratio']
    ratio = ['priceToBookRatio','priceToSalesRatio','priceEarningsRatio','priceToFreeCashFlowsRatio','priceToOperatingCashFlowsRatio','priceCashFlowRatio','grossProfitMargin','operatingProfitMargin','pretaxProfitMargin','netProfitMargin','returnOnEquity','currentRatio','quickRatio','cashRatio','debtEquityRatio', 'eps','PEG Ratio']
    y_name = 'pct_chg'

    # dictionary of hyperparameter values in the lists for grid search
    gs_params = {'colsample_bytree': [0.3, 0.5, 0.7, 0.9],
         'colsample_bylevel': [0.3, 0.5, 0.7, 0.9],
         'eta': [0.005, 0.05, 0.5],
         'max_depth': [3, 5, 7],
         'reg_alpha': [0.0005, 0.005, 0.05],
         'reg_lambda': [0.0005, 0.005, 0.05],
         'subsample': [0.3, 0.5, 0.7, 0.9],
         'min_child_weight': [1, 5, 10], 
         'gamma': [0.0, 0.1, 0.2 , 0.3, 0.4]}



    # converting data to SVMLight format
    dtrain = xgb.DMatrix(train[x_names], train[y_name])
    dvalid = xgb.DMatrix(valid[x_names], valid[y_name])

    # monotonic XGBoost grid search
    best_mxgb = xgb_grid(dtrain, dvalid, gs_params=gs_params, n_models=50, ntree=1000, early_stopping_rounds=100, 
                seed=SEED)

    xgb_prediction = best_mxgb.predict(xgb.DMatrix(train[x_names]))
    
    best_mxgb_phat = pd.DataFrame(best_mxgb.predict(dvalid, iteration_range=(0, best_mxgb.best_ntree_limit)), columns=['phat']) 
    best_mxgb_phat = pd.concat([valid.reset_index(drop=True), best_mxgb_phat], axis=1)
    all_xgb_phat=all_xgb_phat.append(best_mxgb_phat)
    shapley_contribs = best_mxgb.predict(dvalid, pred_contribs=True, iteration_range=(0, best_mxgb.best_ntree_limit))
    global_fi = pd.DataFrame()
    shapley_contribs_df = pd.DataFrame(shapley_contribs, columns=x_names + ['intercept'])
    global_fi['MXGB Importance'] = np.abs(shapley_contribs_df.drop('intercept', axis=1)).mean()
    global_fi['ratio'] = global_fi.index
    im= global_fi[global_fi['ratio'].isin(ratio)].sort_values(by='MXGB Importance', ascending=False)
    im['weights'] = im['MXGB Importance'] / (im['MXGB Importance'].sum())
    im['Quarter'] = j
    weights_xgb= weights_xgb.append(im)


C:\Users\zzxxc\AppData\Local\Temp/ipykernel_12368/789746661.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_q['Ticker'] = lbl.fit_transform(data_q['Ticker'].astype(str))


Grid search run 1/50:
Training with parameters: {'booster': 'gbtree', 'eval_metric': 'rmse', 'nthread': 4, 'objective': 'reg:squarederror', 'seed': 12345, 'colsample_bytree': 0.9, 'colsample_bylevel': 0.9, 'eta': 0.5, 'max_depth': 7, 'reg_alpha': 0.0005, 'reg_lambda': 0.005, 'subsample': 0.3, 'min_child_weight': 5, 'gamma': 0.0}
Grid search new best score discovered at iteration 1/50: 0.0491.
---------- ----------
Grid search run 2/50:
Training with parameters: {'booster': 'gbtree', 'eval_metric': 'rmse', 'nthread': 4, 'objective': 'reg:squarederror', 'seed': 12345, 'colsample_bytree': 0.3, 'colsample_bylevel': 0.3, 'eta': 0.05, 'max_depth': 5, 'reg_alpha': 0.0005, 'reg_lambda': 0.0005, 'subsample': 0.5, 'min_child_weight': 1, 'gamma': 0.2}
---------- ----------
Grid search run 3/50:
Training with parameters: {'booster': 'gbtree', 'eval_metric': 'rmse', 'nthread': 4, 'objective': 'reg:squarederror', 'seed': 12345, 'colsample_bytree': 0.3, 'colsample_bylevel': 0.3, 'eta': 0.5, 'max_dept

---------- ----------
Grid search run 25/50:
Training with parameters: {'booster': 'gbtree', 'eval_metric': 'rmse', 'nthread': 4, 'objective': 'reg:squarederror', 'seed': 12345, 'colsample_bytree': 0.9, 'colsample_bylevel': 0.7, 'eta': 0.005, 'max_depth': 3, 'reg_alpha': 0.0005, 'reg_lambda': 0.05, 'subsample': 0.3, 'min_child_weight': 1, 'gamma': 0.0}
---------- ----------
Grid search run 26/50:
Training with parameters: {'booster': 'gbtree', 'eval_metric': 'rmse', 'nthread': 4, 'objective': 'reg:squarederror', 'seed': 12345, 'colsample_bytree': 0.5, 'colsample_bylevel': 0.9, 'eta': 0.05, 'max_depth': 7, 'reg_alpha': 0.05, 'reg_lambda': 0.05, 'subsample': 0.9, 'min_child_weight': 5, 'gamma': 0.1}
---------- ----------
Grid search run 27/50:
Training with parameters: {'booster': 'gbtree', 'eval_metric': 'rmse', 'nthread': 4, 'objective': 'reg:squarederror', 'seed': 12345, 'colsample_bytree': 0.9, 'colsample_bylevel': 0.7, 'eta': 0.005, 'max_depth': 3, 'reg_alpha': 0.005, 'reg_lambda': 

---------- ----------
Grid search run 49/50:
Training with parameters: {'booster': 'gbtree', 'eval_metric': 'rmse', 'nthread': 4, 'objective': 'reg:squarederror', 'seed': 12345, 'colsample_bytree': 0.3, 'colsample_bylevel': 0.3, 'eta': 0.005, 'max_depth': 3, 'reg_alpha': 0.005, 'reg_lambda': 0.005, 'subsample': 0.7, 'min_child_weight': 5, 'gamma': 0.4}
---------- ----------
Grid search run 50/50:
Training with parameters: {'booster': 'gbtree', 'eval_metric': 'rmse', 'nthread': 4, 'objective': 'reg:squarederror', 'seed': 12345, 'colsample_bytree': 0.7, 'colsample_bylevel': 0.9, 'eta': 0.005, 'max_depth': 5, 'reg_alpha': 0.005, 'reg_lambda': 0.05, 'subsample': 0.7, 'min_child_weight': 5, 'gamma': 0.0}
---------- ----------
Grid search run 1/50:
Training with parameters: {'booster': 'gbtree', 'eval_metric': 'rmse', 'nthread': 4, 'objective': 'reg:squarederror', 'seed': 12345, 'colsample_bytree': 0.9, 'colsample_bylevel': 0.9, 'eta': 0.5, 'max_depth': 7, 'reg_alpha': 0.0005, 'reg_lambda': 

---------- ----------
Grid search run 22/50:
Training with parameters: {'booster': 'gbtree', 'eval_metric': 'rmse', 'nthread': 4, 'objective': 'reg:squarederror', 'seed': 12345, 'colsample_bytree': 0.9, 'colsample_bylevel': 0.9, 'eta': 0.05, 'max_depth': 3, 'reg_alpha': 0.05, 'reg_lambda': 0.05, 'subsample': 0.7, 'min_child_weight': 5, 'gamma': 0.4}
---------- ----------
Grid search run 23/50:
Training with parameters: {'booster': 'gbtree', 'eval_metric': 'rmse', 'nthread': 4, 'objective': 'reg:squarederror', 'seed': 12345, 'colsample_bytree': 0.9, 'colsample_bylevel': 0.3, 'eta': 0.5, 'max_depth': 5, 'reg_alpha': 0.0005, 'reg_lambda': 0.0005, 'subsample': 0.7, 'min_child_weight': 10, 'gamma': 0.1}
---------- ----------
Grid search run 24/50:
Training with parameters: {'booster': 'gbtree', 'eval_metric': 'rmse', 'nthread': 4, 'objective': 'reg:squarederror', 'seed': 12345, 'colsample_bytree': 0.7, 'colsample_bylevel': 0.7, 'eta': 0.05, 'max_depth': 5, 'reg_alpha': 0.005, 'reg_lambda': 

...

---------- ----------
Grid search run 44/50:
Training with parameters: {'booster': 'gbtree', 'eval_metric': 'rmse', 'nthread': 4, 'objective': 'reg:squarederror', 'seed': 12345, 'colsample_bytree': 0.7, 'colsample_bylevel': 0.3, 'eta': 0.005, 'max_depth': 3, 'reg_alpha': 0.005, 'reg_lambda': 0.05, 'subsample': 0.5, 'min_child_weight': 10, 'gamma': 0.4}
---------- ----------
Grid search run 45/50:
Training with parameters: {'booster': 'gbtree', 'eval_metric': 'rmse', 'nthread': 4, 'objective': 'reg:squarederror', 'seed': 12345, 'colsample_bytree': 0.7, 'colsample_bylevel': 0.7, 'eta': 0.05, 'max_depth': 3, 'reg_alpha': 0.0005, 'reg_lambda': 0.005, 'subsample': 0.7, 'min_child_weight': 10, 'gamma': 0.2}
---------- ----------
Grid search run 46/50:
Training with parameters: {'booster': 'gbtree', 'eval_metric': 'rmse', 'nthread': 4, 'objective': 'reg:squarederror', 'seed': 12345, 'colsample_bytree': 0.5, 'colsample_bylevel': 0.9, 'eta': 0.5, 'max_depth': 3, 'reg_alpha': 0.005, 'reg_lambda'

---------- ----------
Grid search run 17/50:
Training with parameters: {'booster': 'gbtree', 'eval_metric': 'rmse', 'nthread': 4, 'objective': 'reg:squarederror', 'seed': 12345, 'colsample_bytree': 0.3, 'colsample_bylevel': 0.5, 'eta': 0.005, 'max_depth': 5, 'reg_alpha': 0.05, 'reg_lambda': 0.005, 'subsample': 0.5, 'min_child_weight': 5, 'gamma': 0.2}
---------- ----------
Grid search run 18/50:
Training with parameters: {'booster': 'gbtree', 'eval_metric': 'rmse', 'nthread': 4, 'objective': 'reg:squarederror', 'seed': 12345, 'colsample_bytree': 0.5, 'colsample_bylevel': 0.7, 'eta': 0.05, 'max_depth': 3, 'reg_alpha': 0.005, 'reg_lambda': 0.0005, 'subsample': 0.5, 'min_child_weight': 5, 'gamma': 0.3}
---------- ----------
Grid search run 19/50:
Training with parameters: {'booster': 'gbtree', 'eval_metric': 'rmse', 'nthread': 4, 'objective': 'reg:squarederror', 'seed': 12345, 'colsample_bytree': 0.5, 'colsample_bylevel': 0.9, 'eta': 0.05, 'max_depth': 7, 'reg_alpha': 0.005, 'reg_lambda':

---------- ----------
Grid search run 41/50:
Training with parameters: {'booster': 'gbtree', 'eval_metric': 'rmse', 'nthread': 4, 'objective': 'reg:squarederror', 'seed': 12345, 'colsample_bytree': 0.5, 'colsample_bylevel': 0.9, 'eta': 0.5, 'max_depth': 7, 'reg_alpha': 0.0005, 'reg_lambda': 0.05, 'subsample': 0.3, 'min_child_weight': 10, 'gamma': 0.4}
---------- ----------
Grid search run 42/50:
Training with parameters: {'booster': 'gbtree', 'eval_metric': 'rmse', 'nthread': 4, 'objective': 'reg:squarederror', 'seed': 12345, 'colsample_bytree': 0.5, 'colsample_bylevel': 0.9, 'eta': 0.05, 'max_depth': 3, 'reg_alpha': 0.005, 'reg_lambda': 0.005, 'subsample': 0.7, 'min_child_weight': 5, 'gamma': 0.1}
---------- ----------
Grid search run 43/50:
Training with parameters: {'booster': 'gbtree', 'eval_metric': 'rmse', 'nthread': 4, 'objective': 'reg:squarederror', 'seed': 12345, 'colsample_bytree': 0.7, 'colsample_bylevel': 0.3, 'eta': 0.5, 'max_depth': 7, 'reg_alpha': 0.05, 'reg_lambda': 0.

---------- ----------
Grid search run 14/50:
Training with parameters: {'booster': 'gbtree', 'eval_metric': 'rmse', 'nthread': 4, 'objective': 'reg:squarederror', 'seed': 12345, 'colsample_bytree': 0.3, 'colsample_bylevel': 0.9, 'eta': 0.005, 'max_depth': 5, 'reg_alpha': 0.05, 'reg_lambda': 0.005, 'subsample': 0.5, 'min_child_weight': 10, 'gamma': 0.3}
---------- ----------
Grid search run 15/50:
Training with parameters: {'booster': 'gbtree', 'eval_metric': 'rmse', 'nthread': 4, 'objective': 'reg:squarederror', 'seed': 12345, 'colsample_bytree': 0.7, 'colsample_bylevel': 0.3, 'eta': 0.05, 'max_depth': 7, 'reg_alpha': 0.005, 'reg_lambda': 0.005, 'subsample': 0.5, 'min_child_weight': 5, 'gamma': 0.2}
---------- ----------
Grid search run 16/50:
Training with parameters: {'booster': 'gbtree', 'eval_metric': 'rmse', 'nthread': 4, 'objective': 'reg:squarederror', 'seed': 12345, 'colsample_bytree': 0.3, 'colsample_bylevel': 0.7, 'eta': 0.5, 'max_depth': 3, 'reg_alpha': 0.0005, 'reg_lambda':

---------- ----------
Grid search run 38/50:
Training with parameters: {'booster': 'gbtree', 'eval_metric': 'rmse', 'nthread': 4, 'objective': 'reg:squarederror', 'seed': 12345, 'colsample_bytree': 0.7, 'colsample_bylevel': 0.3, 'eta': 0.5, 'max_depth': 5, 'reg_alpha': 0.05, 'reg_lambda': 0.05, 'subsample': 0.5, 'min_child_weight': 5, 'gamma': 0.3}
---------- ----------
Grid search run 39/50:
Training with parameters: {'booster': 'gbtree', 'eval_metric': 'rmse', 'nthread': 4, 'objective': 'reg:squarederror', 'seed': 12345, 'colsample_bytree': 0.9, 'colsample_bylevel': 0.5, 'eta': 0.05, 'max_depth': 3, 'reg_alpha': 0.0005, 'reg_lambda': 0.0005, 'subsample': 0.3, 'min_child_weight': 5, 'gamma': 0.1}
---------- ----------
Grid search run 40/50:
Training with parameters: {'booster': 'gbtree', 'eval_metric': 'rmse', 'nthread': 4, 'objective': 'reg:squarederror', 'seed': 12345, 'colsample_bytree': 0.9, 'colsample_bylevel': 0.7, 'eta': 0.5, 'max_depth': 3, 'reg_alpha': 0.0005, 'reg_lambda': 0

In [59]:
# viewing 'weights_xgb'
weights_xgb


,MXGB Importance,ratio,weights,Quarter
priceToOperatingCashFlowsRatio,0.004001,priceToOperatingCashFlowsRatio,0.118480,2012Q1
currentRatio,0.003527,currentRatio,0.104461,2012Q1
pretaxProfitMargin,0.003458,pretaxProfitMargin,0.102416,2012Q1
returnOnEquity,0.002953,returnOnEquity,0.087441,2012Q1
priceToBookRatio,0.002626,priceToBookRatio,0.077777,2012Q1
...,...,...,...,...
debtEquityRatio,0.001137,debtEquityRatio,0.039719,2022Q1
priceToSalesRatio,0.000454,priceToSalesRatio,0.015859,2022Q1
netProfitMargin,0.000335,netProfitMargin,0.011691,2022Q1
priceCashFlowRatio,0.000248,priceCashFlowRatio,0.008656,2022Q1


In [20]:
# saving the data in the csv format
weights_xgb.to_csv('xgb_weight_j25.csv')
